In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
from pathlib import Path
from compile_sweep_results import compile_sweep_results

In [4]:
root = Path("/media/nick/hdd021/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/symmetry_breaking/pde/sweeps/")
sweep_name = "sweep01_neutralization_v2"
df = compile_sweep_results(output_dir=root / sweep_name)
print(df.shape)
df.head()

(9928, 26)


/home/nick/projects/killi-tracker/symmetry_breaking/analysis/compile_sweep_results.py:50: FutureWarning:

errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead



,K_NL,K_A,K_R,sigma_L,K_I,mu_L,N_amp,N_sigma,sigma_N,D0_N,...,m,p,q,roi_frac,roi_fold_change,max_fold_change,profile_Activator,profile_Repressor,profile_rho,__source_file__
0,1000.000000,100.000000,1000.000000,0.193070,10.000000,0.001033,517.947468,31.622777,10.0,1.85,...,1,2,2,0.166667,6.473675e+01,1.039201e+01,"[5315.6518554688, 5315.6518554688, 5315.651855...","[179.9985656738, 179.9985656738, 179.998565673...","[0.9995322227000001, 0.9995328784, 0.999533712...",result_257e41b5.json
1,37.275937,193.069773,1000.000000,2.682696,37.275937,0.002202,71.968567,100.000000,10.0,1.85,...,1,2,2,0.503291,3.869185e-61,2.066616e-61,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[9.880279810999999e-34, 9.881263368999999e-34,...","[4.5232e-06, 4.5232e-06, 4.5232e-06, 4.5232e-0...",result_90202387.json
2,1000.000000,372.759372,37.275937,5.179475,138.949549,0.001033,71.968567,100.000000,10.0,1.85,...,1,2,2,0.166667,1.421089e+02,7.044864e+01,"[5063.75390625, 5063.75390625, 5063.75390625, ...","[3682.8332519531, 3682.8332519531, 3682.833251...","[0.9882885814000001, 0.9884253740000001, 0.988...",result_ff0545f5.json
3,71.968567,719.685673,71.968567,10.000000,268.269580,0.000485,193.069773,10.000000,10.0,1.85,...,1,2,2,0.515201,0.000000e+00,0.000000e+00,"[5.045143632000001e-17, 5.105274368e-17, 5.226...","[8.656e-07, 8.657e-07, 8.658e-07, 8.6610000000...","[4.5232e-06, 4.5232e-06, 4.5232e-06, 4.5232e-0...",result_35ccf384.json
4,1000.000000,5179.474679,10.000000,2.682696,37.275937,0.004693,193.069773,10.000000,10.0,1.85,...,1,2,2,0.509712,4.557481e-131,2.795448e-132,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.5232e-06, 4.5232e-06, 4.5232e-06, 4.5232e-0...",result_efdc1104.json


In [5]:
fig = px.scatter(df, x="roi_frac", y="roi_fold_change", log_y=True)
# add a line at y=1
fig.add_hline(y=1, line_dash="dash", line_color="white", annotation_text="y=1", annotation_position="top left")
# add vertical line at x=500/3500
fig.add_vline(x=500/3500, line_dash="dash", line_color="white", annotation_text="x=500/3500", annotation_position="top right")
# set axis limits
fig.update_xaxes(title_text="Fraction of ROI", range=[0, 1])
fig.update_yaxes(title_text="Max Fold Change", range=[-4, 4])

fig.show()

In [6]:
df["sim_id"] = df["__source_file__"].str.replace("result_", "").str.replace(".json", "")

In [7]:
param_cols = ["sim_id", 'K_NL', 'K_A', 'K_R', 'K_I', 'mu_L', 'N_amp', 'N_sigma', 'sigma_N',
       'sigma_L', 'D0_N', 'D0_L', 'alpha_L',
       'alpha_N', 'tau_rho', 'n', 'm', 'p', 'q', 'roi_frac', 'roi_fold_change',
       'max_fold_change']
param_df = df[param_cols].copy()

nodal_df = df[["sim_id", 'profile_Activator']].copy()
lefty_df = df[["sim_id", 'profile_Repressor']].copy()
rho_df = df[["sim_id", 'profile_rho']].copy()

# save
param_df.to_csv(root / f"{sweep_name}_params_df.csv", index=False)
nodal_df.to_csv(root / f"{sweep_name}_nodal_df.csv", index=False)
lefty_df.to_csv(root / f"{sweep_name}_lefty_df.csv", index=False)
rho_df.to_csv(root / f"{sweep_name}_rho_df.csv", index=False)

In [12]:
ind = np.where((df["max_fold_change"] > 10) & (df["roi_frac"] > 0.9))[0]
df.loc[ind, :].head()

,K_NL,K_A,K_R,sigma_L,K_I,mu_L,N_amp,N_sigma,sigma_N,D0_N,...,p,q,roi_frac,roi_fold_change,max_fold_change,profile_Activator,profile_Repressor,profile_rho,__source_file__,sim_id
586,37.275937,372.759372,37.275937,5.179475,10.000000,0.000485,517.947468,31.622777,10.0,1.85,...,2,2,0.973194,94.304247,21.465248,"[3.6e-09, 3.8e-09, 4.1e-09, 4.7e-09, 5.4e-09, ...","[5.8839550018, 5.9029717445, 5.9410672188, 5.9...","[4.5232e-06, 4.5232e-06, 4.5232e-06, 4.5232e-0...",result_4fe1390e.json,4fe1390e
1147,71.968567,100.000000,71.968567,10.000000,37.275937,0.000485,193.069773,100.000000,10.0,1.85,...,2,2,0.989216,45.275610,49.802210,"[8e-10, 8e-10, 9e-10, 1e-09, 1.2e-09, 1.400000...","[5.3760595322, 5.3934426308, 5.4282646179, 5.4...","[4.5232e-06, 4.5232e-06, 4.5232e-06, 4.5232e-0...",result_5c94b3a3.json,5c94b3a3
1192,71.968567,193.069773,1000.000000,5.179475,37.275937,0.000228,10.000000,100.000000,10.0,1.85,...,2,2,0.934108,11.699103,10.226420,"[2.815e-07, 2.882e-07, 3.017e-07, 3.2240000000...","[2.5481526852, 2.5518693924, 2.5593087673, 2.5...","[4.5232e-06, 4.5232e-06, 4.5232e-06, 4.5232e-0...",result_bb042bda.json,bb042bda
1363,1000.000000,719.685673,10.000000,10.000000,71.968567,0.000228,517.947468,100.000000,10.0,1.85,...,2,2,0.959992,41.393014,37.329137,"[2.85833e-05, 2.92195e-05, 3.0505800000000002e...","[108.9579696655, 109.1226348877, 109.452217102...","[4.5232e-06, 4.5232e-06, 4.5232e-06, 4.5232e-0...",result_075b4b62.json,075b4b62
2103,71.968567,100.000000,10.000000,10.000000,19.306977,0.000485,193.069773,31.622777,10.0,1.85,...,2,2,0.992599,115.142701,47.866121,"[4e-10, 4e-10, 5e-10, 6e-10, 6e-10, 8e-10, 9e-...","[4.0941233635, 4.107360363, 4.1338768005, 4.17...","[4.5232e-06, 4.5232e-06, 4.5232e-06, 4.5232e-0...",result_7dedcc60.json,7dedcc60


In [13]:
nodal_profile = np.asarray(nodal_df.loc[ind[0], "profile_Activator"])
fig = px.scatter(x=np.arange(len(nodal_profile)), y=nodal_profile)
fig.show()

In [46]:
param_df.loc[ind]

,sim_id,K_NL,K_A,K_R,K_I,mu_L,N_amp,N_sigma,sigma_N,sigma_L,...,alpha_L,alpha_N,tau_rho,n,m,p,q,roi_frac,roi_fold_change,max_fold_change
9002,98e30a02,77.426368,2154.43469,46.415888,359.381366,0.000293,10000.0,100.0,10.0,0.1,...,0.46,4.495556,3600,2,1,2,2,0.851788,3.680017,1.826367


In [47]:
np.unique(param_df["mu_L"])

array([5.0118700e-05, 9.0272500e-05, 1.6259650e-04, 2.9286450e-04,
       5.2749970e-04, 9.5011850e-04, 1.7113283e-03, 3.0823992e-03,
       5.5519359e-03, 1.0000000e-02])